In [2]:
import io
import torch
import torch.onnx
from model.LPRNet_noname import build_lprnet

pthfile =r'./save_model/lprnet84.0.pth'

lprnet = build_lprnet(lpr_max_len=8, phase=False, class_num=66, dropout_rate=0.5)
lprnet = torch.load(pthfile, map_location='cpu')

dummy_input = torch.randn(1, 3, 24, 94)
input_names = [ 'input' ]
output_names = [ 'output' ]

onnxfile = pthfile[0:-3]+ r'onnx'
torch.onnx.export(lprnet, 
                  dummy_input, 
                  onnxfile, 
                  verbose=True,
                  export_params=True, #
                  input_names=input_names, 
                  output_names=output_names)

/home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'model.LPRNet_noname.LPRNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original s

graph(%input : Float(1, 3, 24, 94),
      %backbone.0.weight : Float(64, 3, 3, 3),
      %backbone.0.bias : Float(64),
      %backbone.3.block.0.weight : Float(32, 64, 1, 1),
      %backbone.3.block.0.bias : Float(32),
      %backbone.3.block.2.weight : Float(32, 32, 3, 1),
      %backbone.3.block.2.bias : Float(32),
      %backbone.3.block.4.weight : Float(32, 32, 1, 3),
      %backbone.3.block.4.bias : Float(32),
      %backbone.3.block.6.weight : Float(128, 32, 1, 1),
      %backbone.3.block.6.bias : Float(128),
      %backbone.6.block.0.weight : Float(64, 64, 1, 1),
      %backbone.6.block.0.bias : Float(64),
      %backbone.6.block.2.weight : Float(64, 64, 3, 1),
      %backbone.6.block.2.bias : Float(64),
      %backbone.6.block.4.weight : Float(64, 64, 1, 3),
      %backbone.6.block.4.bias : Float(64),
      %backbone.6.block.6.weight : Float(256, 64, 1, 1),
      %backbone.6.block.6.bias : Float(256),
      %backbone.8.block.0.weight : Float(64, 256, 1, 1),
      %backbone.8.bl

In [8]:
print(lprnet)

LPRNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
    (3): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (5): ReLU()
        (6): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (4): ReLU()
    (5): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2), padding=0, dilation=1, ceil_mode=False)
    (6): small_basic_block(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): ReLU()
        (4): Conv2d(64, 6

In [9]:
!pwd

/home/luxian/tf/projects/LPRNet/LPRNet_Pytorch-master


In [7]:
from onnx_tf.backend import prepare
import onnx

# onnx to pb
TF_PATH = "./save_model/pbmodel" # where the representation of tensorflow model will be stored
ONNX_PATH = r'./save_model/lprnet81.0.onnx' # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model
tf_rep = prepare(onnx_model)  # creating TensorflowRep object
tf_rep.export_graph(TF_PATH)

ValueError: in user code:

    /home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/onnx_tf/backend_tf_module.py:98 __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    /home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/onnx_tf/backend.py:289 _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    /home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/onnx_tf/handlers/handler.py:59 handle  *
        return ver_handle(node, **kwargs)
    /home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/onnx_tf/handlers/backend/max_pool.py:28 version_8  *
        return cls._common(node, **kwargs)
    /home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/onnx_tf/handlers/backend/max_pool.py:20 _common  *
        kwargs.get("strict", True))
    /home/luxian/anaconda3/envs/pth-tf/lib/python3.6/site-packages/onnx_tf/handlers/backend/pool_mixin.py:171 pool  **
      

In [ ]:
# pb 转 tflite
import tensorflow as tf

TF_PATH = r"./save_model/pbmodel" 
TFLITE_PATH = pthfile[0:-3]+ r'tflite'
converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_model = converter.convert()
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)